In [146]:
#PreProcessing
import json
import pickle
import copy
import sys
import math
#this function remove stop word from text
def Remove_Stopwords(text_tokenized):
    text_without_sw=[token for token in text_tokenized if token.lower() not in stop_words]
    return text_without_sw

#this function Remove_Punctuation from text
def Remove_Punctuation(text_tokenized_wsw):
    text_without_pun=[token for token in text_tokenized_wsw if token not in punctuations]
    return text_without_pun

#this Function coverts letters into lower.
def casefolding(text):
    text_casefolded=[]
    for token in text:
        text_casefolded.append(token.upper())
    return text_casefolded


#this word tokenize the text into words and replace - with space
def word_tokenization(data):
    data=data.lower()
    characters="abcdefghijklmnopqrstuvwxyz"
    special_character=['\n',' ']
    for i in data:
        if i == '-':
            data=data.replace(i," ")
        if i not in characters:
            if i not in special_character:
                data=data.replace(i,"")
    data=data.split()
    return(data)


#this function lemmatize the words
def lemmantization_of_words(text):
    lemmentized_words=[]
    for token in text:
        lemmentized_words.append(lemmatizer.lemmatize(token))
    return lemmentized_words
#finds the frequency of token


def find_frequency_of_token(text_tokenized, token):
    return [position for position, term in enumerate(text_tokenized) if term == token]


#write Document_term_index to file
def Write_Document_term_index_to_file(Document_term_index):
    sorted_Document_term_index = {}
    sorted_Document_term_index_keys = sorted(Document_term_index.keys())  

    for term in sorted_Document_term_index_keys:
        sorted_Document_term_index[term] = Document_term_index[term]
    file = open("Document_term_index.pkl", "wb")
    pickle.dump(sorted_Document_term_index, file)
    file.close()


In [147]:
import pprint
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

file=open("C:\\Users\\Yougesh\\CS317-IR Dataset for A1\\Stopword-List.txt",'r')

stop_words=file.read()

punctuations = '''!()-[]{};:'‘"\,<>./?@#$%^&*“’”_~'''

Document_term_index=dict()
idf_Document_term_index=dict()

list_t=""

for x in range(1,51):
    file=open("C:\\Users\\Yougesh\\CS317-IR Dataset for A1\\ShortStories\\"+str(x)+".txt",'r',encoding="utf8")
    
    data=file.read()
    
    text_without_sw=Remove_Stopwords(word_tokenization(data))
    
    text_cleaned=Remove_Punctuation(text_without_sw)
    
    lemmentized_words=lemmantization_of_words(text_cleaned)
    
    text_casefolded=casefolding(lemmentized_words)
    #Building Document_term_index
    if x==1:
        for token in lemmentized_words:
            Document_term_index[token]={}
            Document_term_index[token][x]=len(find_frequency_of_token(lemmentized_words,token))
    else:
        list_keys=Document_term_index.keys()
        for token in lemmentized_words:
            if token not in list_keys:
                Document_term_index[token]={}
        for token in lemmentized_words:
                if token in list_keys:
                    if x not in Document_term_index.get(token):
                        Document_term_index[token][x]=len(find_frequency_of_token(lemmentized_words,token))
Write_Document_term_index_to_file(Document_term_index)

In [208]:
x=51
file = open("Document_term_index.pkl", "rb")
Document_term_index = pickle.load(file)

#Query Processing
Query=input("Enter the Query:")
text_without_sw=Remove_Stopwords(word_tokenization(Query))    
text_cleaned=Remove_Punctuation(text_without_sw)
lemmentized_words=lemmantization_of_words(text_cleaned)
text_casefolded=casefolding(lemmentized_words)

list_keys=Document_term_index.keys()
for token in lemmentized_words:
    if token not in list_keys:
        Document_term_index[token]={}
for token in lemmentized_words:
    if token in list_keys:
        if x not in Document_term_index.get(token):
            Document_term_index[token][x]=len(find_frequency_of_token(lemmentized_words,token))

#sorting after adding query terms
sorted_Document_term_index = {}
sorted_Document_term_index_keys = sorted(Document_term_index.keys())  
for term in sorted_Document_term_index_keys:
    sorted_Document_term_index[term] = Document_term_index[term]
Document_term_index = copy.deepcopy(sorted_Document_term_index)
list_keys=Document_term_index.keys()

#calculating idf for every term
for term in list_keys:
    if q_doc in Document_term_index.get(term).keys():
        docf=len(Document_term_index.get(term))-1
        if docf!=0:
              idf_Document_term_index[term]=math.log(docf,2)/50
        else:
            idf_Document_term_index[term]=0
          
    else:
        docf=len(Document_term_index.get(term))
        if docf != 0:
            idf_Document_term_index[term]=math.log(docf,2)/50
        else:
            idf_Document_term_index[term]=0

#giving 0 to non-occuredd term in document
for term in list_keys:
    for x in range(1,52):
        if x not in Document_term_index.get(term):
            Document_term_index[term][x]=0

            
            
#calculating idf*tf
for term in list_keys:
    for x in range(1,52):
        Document_term_index[term][x]=Document_term_index[term][x]*idf_Document_term_index[term]


#Building up vectors
vectorlists=dict()
vector=[]
for x in range(1,52):
    for term in Document_term_index.keys():
        vector.append(Document_term_index[term][x])
    vectorlists[x]=vector
    vector=[]


#finding mod of the document vectors and query vector
m_doc_vectors=dict()
sum_1=0
for key in vectorlists.keys():
    for value in vectorlists.get(key):
        sum_1=sum_1+(value**2)
    m_doc_vectors[key]=math.sqrt(sum_1)
    sum_1=0

#finding similiratiy between vectors
result_set=dict()
results=[]
alpha=0.0061


for key in vectorlists.keys():
    for value,value1 in zip(vectorlists.get(key),vectorlists.get(51)):
        sum_1=sum_1+(value*value1)
    if (m_doc_vectors.get(key)*m_doc_vectors.get(51))==0:
        result_set[key]=0
    else:
        result_set[key]=sum_1/(m_doc_vectors.get(key)*m_doc_vectors.get(51))
    sum_1=0

#filtering results greater than alpha
for (key,value) in result_set.items():
    if key !=51:
        if value >= alpha:
            results.append(key)

print("Result:"+str(results))
print("Length : "+str(len(results)))

Enter the Query:picked news phrases
Result:[15, 20, 22, 40]
Length : 4
